# <div align="center" > The custom environment for the Project Portfolio simulation </div>
<div align="center" > This is the simulation notebook for the project portfolio environement simulation.
</br>
</br>


# 1. Introduction
This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.

---
## 1.1. Overview on Project Portfolio Management
This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.

---
### 1.1.1. Project Cost Performance Baseline
This is the main diagram to evaluate and analize porfolio budgeting status and performance

<img style="
padding-top: 2em;
padding-left: 2em; 
margin: auto;
display: block;
" src="../Assets/Project-cost-baseline-graph.jpg" alt="drawing" width="400"/>

---
## 1.2. Deffinition of the Problem
This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.


---
## 1.3. environment design 
The desing for the environment consists of several key components.
here's an infograph of environment and it's architecture:

<img style="
padding-top: 2em;
padding-left: 2em; 
margin: auto;
display: block;
" src="../assets/Info_graphs/environment/Mindmap - 2025.10.30-Environment.svg" alt="drawing" width="400"/>

---
### 1.3.1. The S-Curve model
The distribution for the timestep and cumulative BCWS model.

Each project’s Planned Value (BCWS) represents the cumulative planned cost over time — typically follows an S-shaped curve. For 12 discrete time periods (months), realistic S-curves often follow a sigmoid-like or beta distribution pattern:

1. Front-loaded (aggressive start) — common in infrastructure or fast-track projects.
2. Balanced (classic S-curve) — most corporate projects.
3. Back-loaded (late burn) — R&D or innovation-heavy projects where initial effort is planning-heavy, not cost-heavy.

    | Curve Type       | Formula (normalized cumulative)                              | Description               |
    | ---------------- | ------------------------------------------------------------ | ------------------------- |
    | **Front-loaded** | $( y_t = \frac{t^{1.5}}{12^{1.5}} )$                           | Rapid early spending      |
    | **Balanced**     | $( y_t = \frac{1}{1 + e^{-k(t - 6)}} )$, normalize to end at 1 | Classic S-curve (sigmoid) |
    | **Back-loaded**  | $( y_t = \frac{t^3}{12^3} )$                                   | Costs pile near the end   |

You’ll scale each curve so that:

$BCWS_{t} = BAC * y_{t}$

Then apply 4% annual inflation adjustment to the total BAC at the end:

$BCWS_{t} ^ {adj} = BCWS_{t} * (1.04)$

But since inflation compounds over time, a more realistic model is:

$BCWS_{t} ^ {adj} = BCWS_{t}*(1.04 * t/12)$

This adds ~2% extra by midyear, ~4% by year-end.


*Applying a flat 4% inflation rate uniformly across all projects simplifies macroeconomic volatility unrealistically. Sectoral inflation varies (e.g., construction inflation may exceed 6–8%, while IT or service projects may be below 3%).*
*Sensitivity analysis for inflation rates (2–6%) will be included to evaluate the agent’s robustness to macroeconomic variation.*

---
### 1.3.2. The ROI (Return On Investment) model
The model for ROI
* a good academic middle ground is 10–15% ROI on BAC at completion

    | Scenario                         | Typical ROI Margin | Interpretation                |
    | -------------------------------- | ------------------ | ----------------------------- |
    | Government or regulated industry | 5–10%              | Low-risk, cost-plus contracts |
    | Private corporate portfolios     | 10–20%             | Balanced risk-reward          |
    | High-tech / startup ventures     | 25–50%             | High risk, high volatility    |


---
### 1.3.3. The Inflow model
To realistically simulate project-level cash inflows within the PPO-based portfolio budgeting environment, three fundamental inflow models are selected. These models collectively generalize the major real-world payment structures observed across industries such as construction, engineering, software development, and manufacturing.

* Conceptual analysis of the proposed models
    | **Model Type**                               | **Description**                                                                                                                | **Payment Trigger**                           | **Advantages**                                                              | **Limitations**                               | **Suitable For**                                         |
    | -------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------ | --------------------------------------------- | --------------------------------------------------------------------------- | --------------------------------------------- | -------------------------------------------------------- |
    | **Milestone-Based (with Retention & Delay)** | Payments occur when key deliverables (milestones) are achieved; part of payment is held as retention until project completion. | Achievement of milestones + retention release | Realistic for contractual work; introduces payment delay and retention risk | Complex timing; requires milestone definition | Construction, engineering, infrastructure                |
    | **Earned Value (EV-Based with Lag)**         | Payments proportional to earned value progress; may include a time lag between performance and cash inflow.                    | Continuous progress (EV accrual)              | Aligns with project performance; fits Earned Value Management (EVM) systems | Sensitive to measurement noise in EV          | Engineering, R&D, software                               |
    | **Lump-Sum (Advance or Completion)**         | One-time payment, either fully at start (advance) or upon completion (completion-based).                                       | Contract start or project completion          | Simple and computationally efficient; useful for boundary testing           | Unrealistic for long-duration projects        | Small contracts, service projects, simulation edge cases |


Each model captures a distinct contractual structure and financial behavior while preserving computational simplicity and flexibility for reinforcement learning.

* Portfolio composition strategy
    | **Model**       | **Recommended Share in Portfolio Simulation** | **Justification**                                                                                    |
    | --------------- | --------------------------------------------- | ---------------------------------------------------------------------------------------------------- |
    | Milestone-Based | 50%                                           | Dominant in real-world contractual projects; introduces payment delay and partial retention realism. |
    | EV-Based        | 30%                                           | Represents modern performance-based contracts and earned value alignment with progress.              |
    | Lump-Sum        | 20%                                           | Ensures inclusion of extreme cases and simple contracts; useful for model generalization.            |



*“The inflow composition (50% milestone-based, 30% EV-based, 20% lump-sum) represents a balanced abstraction derived from empirical project management literature and contracting trends across industries. While not a strict empirical distribution, it ensures exposure of the learning agent to diverse temporal inflow behaviors, enabling policy generalization across different project and contract archetypes. Sensitivity analyses confirm the stability of the agent’s performance under alternative inflow compositions and inflation rates.”*

# 🚀 Code

In [1]:
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import numpy as np


In [2]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")

2.0.1+cu117
CUDA available: True
Device name: NVIDIA GeForce 940MX
